# Descriptive statistics on simulated results
Below is a compact, Jupyter-friendly tutorial for creating descriptive statistics from apsimNGpy simulation outputs—plus an “alternate print” path using tabulate (remember to install it first).

# Prerequisites

# core need libraries
pip install apsimNGpy pandas

# optional pretty printing (used later)
pip install tabulate


Firs fetch the simulated output as a Pandas data frame


In [1]:
import pandas as pd
from apsimNGpy.core.apsim import ApsimModel

# Load a built-in template; omit the .apsimx suffix for templates
model = ApsimModel('Maize')

# If your file hasn’t been run yet, you may need:
model.run()

# Pull the “Report” table as a pandas DataFrame
df = model.get_simulated_output('Report')  # NOTE: correct quotes

# Optional: parse dates if present
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df.head()


,CheckpointID,SimulationID,Zone,Clock.Today,Maize.Phenology.CurrentStageName,Maize.AboveGround.Wt,Maize.AboveGround.N,Yield,Maize.Grain.Wt,Maize.Grain.Size,Maize.Grain.NumberFunction,Maize.Grain.Total.Wt,Maize.Grain.N,Maize.Total.Wt,source_table
0,1,1,Field,1991-05-28 00:00:00,HarvestRipe,1603.309641,15.578250,8469.615813,846.961581,0.278267,3043.698222,846.961581,11.178291,1728.427114,Report
1,1,1,Field,1992-04-09 00:00:00,HarvestRipe,849.734144,9.684291,4674.514452,467.451445,0.273804,1707.246422,467.451445,6.226327,922.393712,Report
2,1,1,Field,1993-03-16 00:00:00,HarvestRipe,182.766781,1.861545,555.021350,55.502135,0.304067,182.532674,55.502135,0.752357,204.108770,Report
3,1,1,Field,1994-03-15 00:00:00,HarvestRipe,795.133784,8.344360,3504.274669,350.427467,0.226733,1545.553056,350.427467,4.886844,869.242545,Report
4,1,1,Field,1995-04-04 00:00:00,HarvestRipe,1525.129268,16.481579,7820.119109,782.011911,0.273512,2859.155304,782.011911,10.463854,1665.483701,Report


Whole-dataset descriptive statistics

In [2]:
# Select numeric columns only
num = df.select_dtypes('number')

# Basic summary
basic = num.agg(['count', 'mean', 'std', 'min', 'median', 'max']).T

# Add CV% and IQR
q = num.quantile([0.25, 0.75]).T.rename(columns={0.25: 'q25', 0.75: 'q75'})
summary = (
    basic
    .join(q)
    .assign(cv_percent=lambda x: (x['std'] / x['mean']) * 100)
)[['count','mean','std','cv_percent','min','q25','median','q75','max']]

summary.round(3)


,count,mean,std,cv_percent,min,q25,median,q75,max
CheckpointID,10.0,1.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000
SimulationID,10.0,1.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000
Maize.AboveGround.Wt,10.0,1225.100,561.129,45.803,182.767,840.745,1318.364,1587.793,1968.461
Maize.AboveGround.N,10.0,12.381,5.400,43.611,1.862,8.679,13.336,16.461,18.309
Yield,10.0,5636.530,2895.961,51.378,555.021,3578.665,6033.993,8239.976,8823.516
Maize.Grain.Wt,10.0,563.653,289.596,51.378,55.502,357.866,603.399,823.998,882.352
Maize.Grain.Size,10.0,0.285,0.024,8.339,0.227,0.275,0.295,0.299,0.305
Maize.Grain.NumberFunction,10.0,1986.771,992.188,49.940,182.533,1339.924,2123.083,2846.481,3043.698
Maize.Grain.Total.Wt,10.0,563.653,289.596,51.378,55.502,357.866,603.399,823.998,882.352
Maize.Grain.N,10.0,7.459,3.754,50.331,0.752,4.926,7.989,10.927,11.254


# Grouped descriptive statistics (e.g., by simulation or treatment)
For this example, we need the `ExperimentManager`` Class


In [11]:
from apsimNGpy.core.config import apsim_version
from apsimNGpy import version
from apsimNGpy.settings import logger
logger.info(f"Notebook generated by;\n APSIM version: `{apsim_version()}`\n apsimNGpy version {version.version}")

2025-10-05 16:42:33,216 - C:\Users\rmagala/apsimNGpy_sim.log - INFO - Notebook generated by;
 APSIM version: `APSIM 2025.8.7844.0`
 apsimNGpy version 0.39.9.15
